In [45]:
import torch
from torch import Tensor
from nba_api.stats.static import teams
from pandas import DataFrame
from typing import List, Tuple

In [2]:
all_teams = teams.get_teams()

In [3]:
team = [team for team in all_teams if team['abbreviation'] == 'ATL'][0]
team_id = team['id']
print(team_id)

1610612737


In [4]:
from nba_api.stats.endpoints import leaguegamefinder
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
games = gamefinder.get_data_frames()
print(games[0].head())

  SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
0     22025  1610612737               ATL  Atlanta Hawks  0022500775   
1     22025  1610612737               ATL  Atlanta Hawks  0022500765   
2     22025  1610612737               ATL  Atlanta Hawks  0022500751   
3     22025  1610612737               ATL  Atlanta Hawks  0022500735   
4     22025  1610612737               ATL  Atlanta Hawks  0022500720   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2026-02-11    ATL @ CHA  L  239  107  ...   0.767     8    33   41   28   
1  2026-02-09    ATL @ MIN  L  240  116  ...   0.786    17    30   47   20   
2  2026-02-07  ATL vs. CHA  L  239  119  ...   0.882     5    25   30   30   
3  2026-02-05  ATL vs. UTA  W  241  121  ...   0.739    19    31   50   34   
4  2026-02-03    ATL @ MIA  W  240  127  ...   0.615     8    39   47   34   

    STL  BLK  TOV  PF  PLUS_MINUS  
0   8.0    6   13  21        -3.0  
1  12.0    1   18  19     

In [5]:
# takes in the id of team A and the abbreviation of team B and returns their 5 latest matchups
def last_five_games_against_b(a_id: str, b_abr: str):
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=a_id)
    a_games = gamefinder.get_data_frames()[0]
    return a_games[a_games.MATCHUP.str.contains(b_abr)].head()


In [6]:
matchups = last_five_games_against_b(team_id, 'CHA')
matchups

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612737,ATL,Atlanta Hawks,0022500775,2026-02-11,ATL @ CHA,L,239,107,...,0.767,8,33,41,28,8.0,6,13,21,-3.0
2,22025,1610612737,ATL,Atlanta Hawks,0022500751,2026-02-07,ATL vs. CHA,L,239,119,...,0.882,5,25,30,30,9.0,6,10,20,-7.0
28,22025,1610612737,ATL,Atlanta Hawks,0022500371,2025-12-18,ATL @ CHA,L,238,126,...,0.864,9,26,35,31,6.0,2,15,20,-7.0
38,22025,1610612737,ATL,Atlanta Hawks,0022500276,2025-11-23,ATL vs. CHA,W,242,113,...,0.750,10,27,37,32,6.0,5,7,21,3.0
75,22024,1610612737,ATL,Atlanta Hawks,0022400993,2025-03-18,ATL @ CHA,W,241,134,...,0.800,11,35,46,27,12.0,8,10,17,32.0


In [7]:
def season_from_id(season_id: str):
    start_year = season_id[-2:]
    end_year = str(int(start_year)+1)
    return f'20{start_year}-{end_year}'

In [30]:
# wrapper for empty tensor
def T(shape: tuple) -> torch.Tensor:
    return torch.empty(shape)

In [85]:
# quantities included in input tensor
num_quantities = 20

In [86]:
from nba_api.stats.endpoints import boxscoretraditionalv3
from nba_api.stats.endpoints import commonteamroster

# get box scores of each player for each game in df of games
def get_box_scores(games: DataFrame) -> Tuple[List[List[DataFrame]], str, str, str]:
    box_score_frames = []
    for game in games.itertuples():
        season = season_from_id(game.SEASON_ID) 
        game_id = game.GAME_ID

        team_id = game.TEAM_ID
        opp_abv = game.MATCHUP[-3:]
        opp_id = teams.find_team_by_abbreviation(opp_abv)['id']

        box_score_frames.append(
            boxscoretraditionalv3.BoxScoreTraditionalV3(game_id=game_id).get_data_frames()
        )
    return box_score_frames, team_id, opp_id, season
        

In [97]:
def compile_input_vector(box_scores: List[List[DataFrame]], 
                         a_id: str, b_id: str, season: str,
                         num_quantities) -> Tensor:
    num_games = len(box_scores)
    num_players = len(
        commonteamroster.CommonTeamRoster(team_id=a_id, season=season).get_data_frames()[0]
    ) + len(
        commonteamroster.CommonTeamRoster(team_id=b_id, season=season).get_data_frames()[0]
    )
    input = T((num_games, num_players, num_quantities))
    print(input.shape)
    # for game in [box_score[0] for box_score in box_scores]:
    #     print(game)

        

In [98]:
box_scores, team_id, opp_id, season = get_box_scores(matchups)
compile_input_vector(box_scores, team_id, opp_id, season, num_quantities)

torch.Size([5, 36, 20])
